In [5]:
import os

In [6]:
%pwd

'c:\\Users\\nico0\\OneDrive\\Documentos\\2024\\end_to_end_projects\\cancer_project\\end-to-end-cancer-project-hands-on\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\nico0\\OneDrive\\Documentos\\2024\\end_to_end_projects\\cancer_project\\end-to-end-cancer-project-hands-on'

## 1.Explicación de entidades

la entidad tiene las mismas variables que las definidas en el config.yaml


data_ingestion:
  root_dir: artifacts/data_ingestion
  #de donde se va a descargar
  source_URL: https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing
  #donde se va a guardar los datos descargados
  local_data_file: artifacts/data_ingestion/data.zip
  #donde de va a guardar los datos descomprimiods
  unzip_dir: artifacts/data_ingestion

In [9]:
from dataclasses import dataclass
from pathlib import Path

#frozen es para que no se pueda agregarle más datos
#el decorador dataclass es para tener una clase que no necesite el self ni el init
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## 2. Probando las constantes

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    #clase para crear los directorios
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, #almacenando los paths en la clase
        params_filepath = PARAMS_FILE_PATH):

        #recordnado que el output son tipo config box por eso se pueden llamar config.file
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        #llamando a dentro del archivo config.yml el atributo artifacts_root
        #y lo que hace es crear ese directorio
        create_directories([self.config.artifacts_root])


    #ingestando datos
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        #llamando a config.data_ingestion.root_dir
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir, #por ejemplo este es: artifacts/prepare_base_model
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

## 3. Componentes

In [12]:
import os
import zipfile #descomprimir
import gdown #descargar de drive
from cnnClassifier import logger #log de lo que se hizo
from cnnClassifier.utils.common import get_size

In [13]:
class DataIngestion:

    #tomando la configuración de ConfigurationManager.get_data_ingestion_config
    #porque de aquí vamos a tomar las direcciones de los folders
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url. es el mismo código que está en trials.ipynb
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)

            #registrando lo que se hizo
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            #registrando lo que se hizo
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)




## 4. Ejecutando el pipeline

In [14]:
try:
    #instanciando la clase
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    #ejecutando los metodos
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-14 23:47:55,972: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-14 23:47:55,979: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-14 23:47:55,979: INFO: common: created directory at: artifacts]
[2024-02-14 23:47:55,985: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-14 23:47:55,988: INFO: 1659025976: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=bcc5b31e-3de0-4b1e-ac1c-1361640f3d4f
To: c:\Users\nico0\OneDrive\Documentos\2024\end_to_end_projects\cancer_project\end-to-end-cancer-project-hands-on\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:11<00:00, 4.12MB/s]

[2024-02-14 23:48:08,726: INFO: 1659025976: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
